<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/GRPO/GRPO_Countdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ⚠️ **Warning: Please note that running the GRPO training job in this notebook to completion has an estimated cost of over $300**.

GRPO training runs for 1000 steps by default. You can stop your training job early via the UI after you're satisfied with seeing the performance. (For example after 50-100 steps is where you can start to see the model learning the rewards.)

# Environment Set Up

In [ ]:
! pip uninstall -y numpy --quiet  # Temporary workaround for the incompatible default numpy version
! pip install predibase
! pip install datasets
! pip install --upgrade datasets huggingface_hub

In [1]:
!python --version

Python 3.11.12


## Note

If you run into a numpy error creating the Predibase client in the cell below, restart the CoLab session (Runtime->Restart Session in the toolbar) and re-run the cells above.

In [ ]:
from predibase import *

pb = Predibase(api_token = "") # api-key

# Data Preprocessing

In [2]:
template = """<|im_start|>system
You are a helpful assistant. You first think about the reasoning process step by step and then provide the user with an answer.<|im_end|>
<|im_start|>user
Using the numbers {nums}, create an equation that equals {target}. You can use basic arithmetic operations (+, -, *, /) and parentheses, and each number can only be used once. Show your work in <think> </think> tags. And return the final equation and answer in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>.<|im_end|>
<|im_start|>assistant
Let me solve this step by step.
<think>"""

In [3]:
def format_row(row):
    return template.format(
        nums=row["nums"],
        target=row["target"]
    )

In [4]:
import pandas as pd

# Load the dataset
# dataset = load_dataset("predibase/countdown") # download_mode=DownloadMode.FORCE_DOWNLOAD
from datasets import load_dataset, DownloadMode

dataset = load_dataset("predibase/countdown", download_mode=DownloadMode.FORCE_REDOWNLOAD)

train_df = pd.DataFrame(dataset["train"])
eval_df = pd.DataFrame(dataset["test"])

README.md:   0%|          | 0.00/438 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.49k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/4.31k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
train_df["prompt"] = train_df.apply(format_row, axis=1)
eval_df["prompt"] = eval_df.apply(format_row, axis=1)

print("train_df")
print(train_df.head(5))
print(eval_df.iloc[0]["prompt"])
print("--------")
print("eval_df")
print(eval_df.head(5))
print(eval_df.iloc[0]["prompt"])

train_df.to_json("./countdown_train.jsonl", lines=True, orient="records")

# Reward Functions
Reward functions are how the model will determine how well it's doing during training. It's important to understand how they are defined.

In [6]:
import regex as re
from typing import List, Dict, Any

# Check if the output is properly formatted
def format_reward_func(prompt: str, completion: str, example: dict[str, str]) -> int:
    # Imported packages must be inside each reward function
    import re

    reward = 0
    try:
        # Add synthetic <think> as it's already part of the prompt and prefilled
        # for the assistant to more easily match the regex
        completion = "<think>" + completion

        # Check if the format matches expected pattern:
        # <think> content </think> followed by <answer> content </answer>
        regex = (
            r"^<think>\s*([^<]*(?:<(?!/?think>)[^<]*)*)\s*<\/think>\n"
            r"<answer>\s*([\s\S]*?)\s*<\/answer>$"
        )

        # Search for the regex in the completion
        match = re.search(regex, completion, re.DOTALL)
        if match is not None and len(match.groups()) == 2:
            reward = 1.0
    except Exception:
        pass

    print(f"Format reward: {reward}")
    return reward

# Check if the output contains the correct answer
def equation_reward_func(prompt: str, completion: str, example: dict[str, str]) -> int:
    # Imported packages must be inside each reward function
    import re
    import ast

    reward = 0.0
    try:
        # add synthetic <think> as its already part of the prompt and prefilled
        # for the assistant to more easily match the regex
        completion = "<think>" + completion
        match = re.search(r"<answer>\s*([\s\S]*?)\s*<\/answer>", completion)
        if not match:
            print("No answer found in completion. Equation reward: 0.0")
            return 0.0

        # Extract the "answer" part from the completion
        equation = match.group(1).strip()
        # Extract all numbers from the equation
        used_numbers = [int(n) for n in re.findall(r'\d+', equation)]

        # Convert the example["nums"] to a list if it's a string
        # This is common for columns like lists in datasets
        if isinstance(example["nums"], str):
            example["nums"] = ast.literal_eval(example["nums"])

        # Check if all numbers are used exactly once
        if sorted(used_numbers) != sorted(example["nums"]):
            print("Numbers used in equation not the same as in example. Equation reward: 0.0")
            return 0.0

        # Define a regex pattern that only allows numbers, operators, parentheses, and whitespace
        allowed_pattern = r'^[\d+\-*/().\s]+$'
        if not re.match(allowed_pattern, equation):
            print("Equation contains invalid characters. Equation reward: 0.0")
            return 0.0

        # Evaluate the equation with restricted globals and locals
        result = eval(equation, {"__builtins__": None}, {})
        # Check if the equation is correct and matches the ground truth
        if abs(float(result) - float(example["target"])) < 1e-5:
            reward = 1.0
        else:
            print("Equation is incorrect. Equation reward: 0.0")
            return 0.0

    except Exception:
        pass

    print(f"Equation reward: {reward}")
    return reward

Note the following:
- `completions` are the model outputs for a given prompt and `examples` is a list of dicts, with each dict representing a row of the train dataset.
- The reward for each completion is appended to the list `rewards`, so the output is a list of floats with the same length as our dataset
- While the rewards in this case take on the binary values of 0.0 and 1.0, they can take on any float values.

# Fine-Tuning

In [7]:
# Upload a dataset
try:
  dataset = pb.datasets.from_file("./countdown_train.jsonl", name="countdown_train")
except:
  dataset = pb.datasets.get("countdown_train")

In [8]:
# Create a new repo/fetch an existing one
repo = pb.repos.create(name="countdown", description="GRPO Countdown Runs", exists_ok=True)

In [9]:
# Launch the finetuning job!
adapter = pb.adapters.create(
    config=GRPOConfig(
        base_model="qwen2-5-7b-instruct",
        reward_fns=RewardFunctionsConfig(
            functions={
              "format": RewardFunction.from_callable(format_reward_func),
              "answer": RewardFunction.from_callable(equation_reward_func),
            }
        ),
        target_modules=[
            'q_proj', 'v_proj', 'k_proj', 'o_proj',
            'gate_proj', 'up_proj', 'down_proj'
        ],
        train_steps=200,
    ),
    dataset="countdown_train",
    repo=repo,
    description="Countdown!"
)

RuntimeError: Bad request. Response status code 403. Error: {'message': 'GRPO fine-tuning is not available in the free tier. Please upgrade to developer tier or book a demo at https://go.predibase.com/rft-early-access.'}

# Evaluate the Model

In [ ]:
pb.deployments.create(
    name="my-qwen2-5-7b-instruct",
    config=DeploymentConfig(
        base_model="qwen2-5-7b-instruct",
        cooldown_time=600,
        min_replicas=0,
        max_replicas=1
    ),
    description="Created with GRPO Countdown notebook",
)

In [ ]:
adapter_id = "countdown/1" # Change the version number as needed (you can find this in the UI)

lorax_client = pb.deployments.client("my-qwen2-5-7b-instruct")

eval_outputs = []
for i, row in eval_df.iterrows():
  prompt = row["prompt"]
  completion = lorax_client.generate(prompt, adapter_id=adapter_id).generated_text
  eval_outputs.append({"prompt": prompt, "completion": completion})

eval_outputs = pd.DataFrame(eval_outputs)
print(eval_outputs.head(5))

eval_outputs.to_json("./eval_outputs.jsonl", lines=True, orient="records")